In [35]:
import polars as pl
import polars_st as st

In [36]:
bottom_tiles = pl.DataFrame(
    {
        "top": [
            "POLYGON ((0 0, 0 1, -1 1, -1 0, 0 0))",
            "POLYGON ((0 0, 0 1, 1 1, 1 0, 0 0))",
        ]
    }
).select(top=st.from_wkt("top"))

bottom_tiles.rename({"top": "geometry"}).st.to_geopandas().explore()

In [37]:
top_tiles = pl.DataFrame(
    {
        "bottom": [
            "POLYGON ((0.5 0, 0.5 1, -0.5 1, -0.5 0, 0.5 0))",
        ]
    }
).select(bottom=st.from_wkt("bottom"))

top_tiles.rename({"bottom": "geometry"}).st.to_geopandas().explore()

In [38]:
pl.concat(
    [top_tiles.rename({"bottom": "geometry"}), bottom_tiles.rename({"top": "geometry"})]
).st.to_geopandas().explore()

In [39]:
intersection = bottom_tiles.join(top_tiles, how="cross").select(
    pl.col("top").st.intersection(pl.col("bottom")).alias("geometry")
)
intersection.st.to_geopandas().explore()

In [40]:
intersection.select(st.geom("geometry").st.area())

geometry
f64
0.5
0.5


# more complex geometry

In [74]:
bottom_tiles = pl.DataFrame(
    {
        "top_name": ["M", "N", "O", "P"],
        "top": [
            "POLYGON ((0 0, 0 1, -1 1, -1 0, 0 0))",
            "POLYGON ((0 0, 0 1, 1 1, 1 0, 0 0))",
            "POLYGON ((0 0, 0 -1, -1 -1, -1 0, 0 0))",
            "POLYGON ((0 0, 0 -1, 1 -1, 1 0, 0 0))",
        ],
    }
).with_columns(top=st.from_wkt("top"))

top_tiles = pl.DataFrame(
    {
        "bottom_name": ["A", "B", "C"],
        "bottom": [
            # "POLYGON ((-1 0, 0 1, 1 0, 0 -1, -1 0))",
            "POLYGON ((-1 -1, 0 1, 1 -1, -1 -1))",
            "POLYGON ((-1 -1, 0 1, -2 1, -1 -1))",
            "POLYGON ((1 -1, 0 1, 2 1, 1 -1))",
        ],
    }
).with_columns(bottom=st.from_wkt("bottom"))

intersection = bottom_tiles.join(top_tiles, how="cross").select(
    pl.exclude("top", "bottom"), pl.col("top").st.intersection(pl.col("bottom")).alias("geometry")
)
pl.concat(
    [top_tiles[["bottom"]].rename({"bottom": "geometry"}), bottom_tiles[["top"]].rename({"top": "geometry"})]
).st.to_geopandas().explore()

In [79]:
intersection.filter(st.geom("geometry").st.area() != 0).with_columns(st.geom("geometry").st.to_wkt()).to_dict(
    as_series=False
)

{'top_name': ['M', 'M', 'N', 'N', 'O', 'O', 'P', 'P'],
 'bottom_name': ['A', 'B', 'A', 'C', 'A', 'B', 'A', 'C'],
 'geometry': ['POLYGON ((0 0, -0.5 0, 0 1, 0 0))',
  'POLYGON ((0 1, -0.5 0, -1 0, -1 1, 0 1))',
  'POLYGON ((0 1, 0.5 0, 0 0, 0 1))',
  'POLYGON ((1 1, 1 0, 0.5 0, 0 1, 1 1))',
  'POLYGON ((0 -1, -1 -1, -0.5 0, 0 0, 0 -1))',
  'POLYGON ((-1 0, -0.5 0, -1 -1, -1 0))',
  'POLYGON ((0 0, 0.5 0, 1 -1, 0 -1, 0 0))',
  'POLYGON ((1 0, 1 -1, 0.5 0, 1 0))']}

In [76]:
intersection.filter(st.geom("geometry").st.area() != 0).st.to_geopandas().explore()

In [83]:
intersection_area = (
    intersection.filter(st.geom("geometry").st.area() != 0)
    .with_columns(pl.col("geometry").st.area().alias("area"))
    .drop("geometry")
)
intersection_area

top_name,bottom_name,area
str,str,f64
"""M""","""A""",0.25
"""M""","""B""",0.75
"""N""","""A""",0.25
"""N""","""C""",0.75
"""O""","""A""",0.75
"""O""","""B""",0.25
"""P""","""A""",0.75
"""P""","""C""",0.25


In [88]:
# %%timeit
(
    intersection_area.with_columns(
        pl.col("area").sum().over("top_name").alias("top_area"),
        pl.col("area").sum().over("bottom_name").alias("bottom_area"),
    )
)

top_name,bottom_name,area,top_area,bottom_area
str,str,f64,f64,f64
"""M""","""A""",0.25,1.0,2.0
"""M""","""B""",0.75,1.0,1.0
"""N""","""A""",0.25,1.0,2.0
"""N""","""C""",0.75,1.0,1.0
"""O""","""A""",0.75,1.0,2.0
"""O""","""B""",0.25,1.0,1.0
"""P""","""A""",0.75,1.0,2.0
"""P""","""C""",0.25,1.0,1.0
